## 1. Data Preparation

In [1]:
# install packages
installed <- installed.packages()[,"Package"]
packages.required = c("Seurat", "dplyr", "patchwork", "Matrix", "parallel","NMF","SingleCellSignalR")

for (package in packages.required){
    if (!(package %in% installed)) {
     #install.packages(package)
     if (!requireNamespace("BiocManager", quietly = TRUE))
         install.packages("BiocManager")
     
     BiocManager::install(package, update = FALSE)
  }
}

if(!require(devtools)) install.packages("devtools")
devtools::install_github("sqjin/CellChat")
devtools::install_github("Coolgenome/iTALK")
devtools::install_github("BaderLab/CCInx")
devtools::install_github("YUZIXD/scMLnet")

In [3]:
# load packages
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(Matrix))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(iTALK))
suppressPackageStartupMessages(library(CellChat))
suppressPackageStartupMessages(library(SingleCellSignalR))
suppressPackageStartupMessages(library(CCInx))
suppressPackageStartupMessages(library(scMLnet))

# load tool functions
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/run_CellChat.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/run_iTalk.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/run_SingleCellSignalR.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/run_scMLnet.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/run_CCInx.R")

In [4]:
# load single-cell sample data
sampleA <- readRDS("https://zenodo.org/record/6497091/files/GSE122960.rds?download=1")
sampleB <- readRDS("https://zenodo.org/record/6497091/files/GSE128033.rds?download=1")
sampleC <- readRDS("https://zenodo.org/record/6497091/files/GSE135893.rds?download=1")
sampleD <- readRDS("https://zenodo.org/record/6497091/files/GSE136831.rds?download=1")

In [5]:
# normalized data of the sample
str(sampleA@assays$RNA@data)
str(sampleB@assays$RNA@data)
str(sampleC@assays$RNA@data)
str(sampleD@assays$RNA@data)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:15028219] 33 35 45 54 55 59 61 72 86 97 ...
  ..@ p       : int [1:10414] 0 3337 5097 7069 10281 13289 15511 18031 19740 21390 ...
  ..@ Dim     : int [1:2] 33694 10413
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:33694] "RP11-34P13.3" "FAM138A" "OR4F5" "RP11-34P13.7" ...
  .. ..$ : chr [1:10413] "AAACCTGAGCACCGCT-1_2" "AAACCTGCACAGATTC-1_2" "AAACCTGGTAAGGGCT-1_2" "AAACCTGGTCAAACTC-1_2" ...
  ..@ x       : num [1:15028219] 0.686 0.401 1.09 0.401 0.686 ...
  ..@ factors : list()
Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:39789428] 47 59 86 97 153 176 189 197 216 219 ...
  ..@ p       : int [1:20628] 0 1379 2847 7202 8373 9651 13150 14990 16263 18922 ...
  ..@ Dim     : int [1:2] 45068 20627
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:45068] "RP11-34P13.3" "FAM138A" "OR4F5" "RP11-34P13.7" ...
  .. ..$ : chr [1:20627] "AAACCTGAGAGGACGG-1_2" "AAACCTGAGCGTCAAG-1_2" "AAACCTGCAGGA

In [6]:
# cell types and amounts of the sample
table(Idents(sampleA))
table(Idents(sampleB))
table(Idents(sampleC))
table(Idents(sampleD))


 Macrophage  Fibroblast         AT2    Monocyte       Tcell         AT1 
       5892         147        2293         774         645         210 
Endothelial        Mast 
        299         153 


Endothelial         AT2  Macrophage    Monocyte  Fibroblast       Tcell 
       2553        1548        7622        1153        3420        1693 
       Mast         AT1 
       1405        1233 


        AT2         AT1 Endothelial        Mast    Monocyte       Tcell 
       3225          96        3496         360        2113        2829 
 Macrophage  Fibroblast 
      16507         152 


 Macrophage    Monocyte       Tcell        Mast  Fibroblast         AT2 
      60483        6793        5088         572        1443         496 
Endothelial         AT1 
       3379         176 

In [8]:
# load gold standard sample
gold <- read.table("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/data/IPF gold standard.txt", header = TRUE, sep = "\t")
str(gold)
head(gold)

'data.frame':	208 obs. of  4 variables:
 $ source  : chr  "AT1" "AT1" "AT1" "AT1" ...
 $ target  : chr  "Monocyte" "Monocyte" "Monocyte" "Macrophage" ...
 $ ligand  : chr  "TGFB1" "TGFB1" "TGFB1" "TGFB1" ...
 $ receptor: chr  "TGFBR1" "TGFBR2" "TGFBR3" "TGFBR1" ...


,source,target,ligand,receptor
,<chr>,<chr>,<chr>,<chr>
1,AT1,Monocyte,TGFB1,TGFBR1
2,AT1,Monocyte,TGFB1,TGFBR2
3,AT1,Monocyte,TGFB1,TGFBR3
4,AT1,Macrophage,TGFB1,TGFBR1
5,AT1,Macrophage,TGFB1,TGFBR2
6,AT1,Macrophage,TGFB1,TGFBR3


In [9]:
# remove unwanted genes for CellChat, iTalk, SingleCellSignalR, and CCInx
genes <- unique(c(gold$ligand, gold$receptor))
sampleA1 <- subset(sampleA, features = genes) 
sampleB1 <- subset(sampleB, features = genes)
sampleC1 <- subset(sampleC, features = genes) 
sampleD1 <- subset(sampleD, features = genes) 

In [10]:
# remove unwanted genes for scMLnet
rec <- read.table("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/scMLnet_database/RecTF.txt", header = T, sep = "\t")
tf <- read.table("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/scMLnet_database/TFTargetGene.txt", header = T, sep = "\t")
rec <- rec[rec$Receptor == genes,]
tf <- tf[tf$TF == rec$TF,]
genes2 <- unique(c(genes, rec$TF, tf$Targets))
sampleA2 <- subset(sampleA, features = genes2)
sampleB2 <- subset(sampleB, features = genes2)
sampleC2 <- subset(sampleC, features = genes2)
sampleD2 <- subset(sampleD, features = genes2)

Warning message in rec$Receptor == genes:
“longer object length is not a multiple of shorter object length”
Warning message in tf$TF == rec$TF:
“longer object length is not a multiple of shorter object length”


## 2. Cell-cell Interaction Prediction 

In [11]:
# CellChat
CellChatA <- run_CellChat(sampleA1)
CellChatB <- run_CellChat(sampleB1)
CellChatC <- run_CellChat(sampleC1)
CellChatD <- run_CellChat(sampleD1)

Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  Macrophage Fibroblast AT2 Monocyte Tcell AT1 Endothelial Mast 
[1] "Total predicted L-R pairs: 1"


Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  Endothelial AT2 Macrophage Monocyte Fibroblast Tcell Mast AT1 
[1] "Total predicted L-R pairs: 7"


Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  AT2 AT1 Endothelial Mast Monocyte Tcell Macrophage Fibroblast 
[1] "Total predicted L-R pairs: 8"


Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  Macrophage Monocyte Tcell Mast Fibroblast AT2 Endothelial AT1 
[1] "Total predicted L-R pairs: 15"


In [12]:
head(CellChatA)
head(CellChatB)
head(CellChatC)
head(CellChatD)

,source,target,ligand,receptor,prob,pval
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,AT1,Fibroblast,PDGFA,PDGFRB,0.002962098,0


,source,target,ligand,receptor,prob,pval
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Macrophage,Macrophage,TGFB1,TGFBR1,0.0014066582,0
2,Macrophage,Macrophage,TGFB1,TGFBR2,0.0014066582,0
21,Monocyte,Macrophage,TGFB1,TGFBR1,0.0008797044,0
22,Monocyte,Macrophage,TGFB1,TGFBR2,0.0008797044,0
3,Macrophage,Macrophage,CCL3,CCR1,0.0013310644,0
4,Monocyte,Macrophage,CCL3,CCR1,0.0013381925,0


,source,target,ligand,receptor,prob,pval
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,AT1,Fibroblast,PDGFA,PDGFRA,0.012570570,0
2,AT1,Fibroblast,PDGFA,PDGFRB,0.010027325,0
3,Monocyte,Macrophage,CCL3,CCR1,0.002577753,0
4,Macrophage,Macrophage,CCL3,CCR1,0.002839586,0
5,Mast,Monocyte,CSF1,CSF1R,0.005637249,0
6,Fibroblast,Monocyte,CSF1,CSF1R,0.005129437,0


,source,target,ligand,receptor,prob,pval
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Macrophage,Fibroblast,TGFB1,TGFBR1,0.010531960,0
2,Macrophage,Fibroblast,TGFB1,TGFBR2,0.010531960,0
21,Monocyte,Fibroblast,TGFB1,TGFBR1,0.004239185,0
22,Monocyte,Fibroblast,TGFB1,TGFBR2,0.004239185,0
3,Tcell,Fibroblast,TGFB1,TGFBR1,0.015503463,0
31,Tcell,Fibroblast,TGFB1,TGFBR2,0.015503463,0


In [13]:
# iTalk
iTalkA <- run_iTalk(sampleA1, top = 208)
iTalkB <- run_iTalk(sampleB1, top = 208)
iTalkC <- run_iTalk(sampleC1, top = 208)
iTalkD <- run_iTalk(sampleD1, top = 208)


[==========================>--------------------------------------------]  38%

[===================================>-----------------------------------]  50%

[===========================================>---------------------------]  62%

[====================================================>------------------]  75%

[=============================================================>---------]  88%

[=======================================================================] 100%
                                                                              



[1] "Total predicted L-R pairs: 509"
[1] "Choose top L-R pairs: 208"
[1] "Total predicted L-R pairs: 511"
[1] "Choose top L-R pairs: 208"
[1] "Total predicted L-R pairs: 566"
[1] "Choose top L-R pairs: 208"



[========>--------------------------------------------------------------]  12%

[=================>-----------------------------------------------------]  25%

[==========================>--------------------------------------------]  38%

[===================================>-----------------------------------]  50%

[===========================================>---------------------------]  62%

[====================================================>------------------]  75%

[=============================================================>---------]  88%

[=======================================================================] 100%
                                                                              



[1] "Total predicted L-R pairs: 407"
[1] "Choose top L-R pairs: 208"


In [14]:
head(iTalkA)
head(iTalkB)
head(iTalkC)
head(iTalkD)

,source,target,ligand,receptor,source_mean_exprs,target_mean_exprs
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
23,Endothelial,Fibroblast,CTGF,LRP1,1.2286853,0.8454993
22,Endothelial,Macrophage,CTGF,LRP1,1.2286853,0.5840580
2,Fibroblast,Fibroblast,CTGF,LRP1,0.8118479,0.8454993
16,AT1,Fibroblast,CTGF,LRP1,0.6911836,0.8454993
1,Fibroblast,Macrophage,CTGF,LRP1,0.8118479,0.5840580
25,Endothelial,Monocyte,CTGF,LRP1,1.2286853,0.3453390


,source,target,ligand,receptor,source_mean_exprs,target_mean_exprs
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1401,Monocyte,Endothelial,IL1B,IL1R1,1.742668,0.6223556
841,Tcell,Macrophage,CCL4,CCR1,3.764675,0.2709540
14,Fibroblast,Fibroblast,CTGF,LRP1,1.186583,0.7871848
1441,Monocyte,Fibroblast,IL1B,IL1R1,1.742668,0.4734596
851,Tcell,Monocyte,CCL4,CCR1,3.764675,0.2031794
1411,Monocyte,AT2,IL1B,IL1R1,1.742668,0.3727768


,source,target,ligand,receptor,source_mean_exprs,target_mean_exprs
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
18,Endothelial,Fibroblast,CTGF,LRP1,1.0213757,1.703772
12,AT1,Fibroblast,CTGF,LRP1,0.6978218,1.703772
24,Fibroblast,Fibroblast,CTGF,LRP1,0.6749418,1.703772
132,Monocyte,AT1,TGFB1,ITGB6,0.6380866,1.557565
236,Monocyte,AT1,TGFB1,TGFBR2,0.6380866,1.421618
136,Macrophage,AT1,TGFB1,ITGB6,0.5808058,1.557565


,source,target,ligand,receptor,source_mean_exprs,target_mean_exprs
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
134,Tcell,AT1,TGFB1,ITGB6,1.1480619,1.589999
5,Fibroblast,Fibroblast,CTGF,LRP1,1.3232609,1.176309
236,Tcell,Endothelial,TGFB1,TGFBR2,1.1480619,1.199099
237,Tcell,AT1,TGFB1,TGFBR2,1.1480619,1.164342
289,Tcell,Fibroblast,TGFB1,TGFBR3,1.1480619,1.158633
128,Macrophage,AT1,TGFB1,ITGB6,0.8076374,1.589999


In [15]:
# SingleCellSignalR
SCSRA <- run_SingleCellSignalR(sampleA1)
SCSRB <- run_SingleCellSignalR(sampleB1)
SCSRC <- run_SingleCellSignalR(sampleC1)
SCSRD <- run_SingleCellSignalR(sampleD1)

No such file as table_dge_Macrophage.txt in the cluster-analysis folder
No such file as table_dge_Fibroblast.txt in the cluster-analysis folder
No such file as table_dge_AT2.txt in the cluster-analysis folder
No such file as table_dge_Monocyte.txt in the cluster-analysis folder
No such file as table_dge_Tcell.txt in the cluster-analysis folder
No such file as table_dge_AT1.txt in the cluster-analysis folder
No such file as table_dge_Endothelial.txt in the cluster-analysis folder
No such file as table_dge_Mast.txt in the cluster-analysis folder
Paracrine signaling: 
Checking for signaling between cell types
0 No significant interaction found from Macrophage to Fibroblast
0 No significant interaction found from Macrophage to AT2
0 No significant interaction found from Macrophage to Monocyte
0 No significant interaction found from Macrophage to Tcell
0 No significant interaction found from Macrophage to AT1
0 No significant interaction found from Macrophage to Endothelial
0 No significant

No such file as table_dge_Macrophage.txt in the cluster-analysis folder
No such file as table_dge_Monocyte.txt in the cluster-analysis folder
No such file as table_dge_Tcell.txt in the cluster-analysis folder
No such file as table_dge_Mast.txt in the cluster-analysis folder
No such file as table_dge_Fibroblast.txt in the cluster-analysis folder
No such file as table_dge_AT2.txt in the cluster-analysis folder
No such file as table_dge_Endothelial.txt in the cluster-analysis folder
No such file as table_dge_AT1.txt in the cluster-analysis folder
Paracrine signaling: 
Checking for signaling between cell types
0 No significant interaction found from Macrophage to Monocyte
0 No significant interaction found from Macrophage to Tcell
0 No significant interaction found from Macrophage to Mast
0 No significant interaction found from Macrophage to Fibroblast
0 No significant interaction found from Macrophage to AT2
0 No significant interaction found from Macrophage to Endothelial
0 No significan

ERROR: Error in `[.data.frame`(data.frame(signal[i]), , c(1, 2, 4)): undefined columns selected


In [16]:
head(SCSRA)
head(SCSRB)
head(SCSRC)
head(SCSRD)

,source,target,ligand,receptor,LRscore
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Fibroblast,Monocyte,CCL2,CCR2,0.68742187149298
2,AT1,Fibroblast,PDGFA,PDGFRB,0.761042984780823
3,AT1,Fibroblast,PDGFC,PDGFRB,0.689645362246055
4,Mast,Fibroblast,TGFB1,TGFBR3,0.598945955404346
5,Mast,Monocyte,CCL2,CCR2,0.548699786843868
6,Mast,Endothelial,TGFB1,TGFBR3,0.718179341579129


,source,target,ligand,receptor,LRscore
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Endothelial,Monocyte,CCL2,CCR2,0.595674932115517
2,Mast,Macrophage,CSF1,CSF1R,0.662648748227722
3,Mast,Monocyte,CSF1,CSF1R,0.676553266700939


,source,target,ligand,receptor,LRscore
,<chr>,<chr>,<chr>,<chr>,<chr>
1,AT1,Fibroblast,PDGFA,PDGFRA,0.760414319142841
2,AT1,Fibroblast,PDGFA,PDGFRB,0.732949670274543
3,AT1,Fibroblast,PDGFC,PDGFRA,0.50380082682914
4,Mast,AT2,TGFB1,ITGB6,0.755698094707322
5,Mast,AT1,TGFB1,ITGB6,0.797994657314319
6,Monocyte,Tcell,CCL3,CCR4,0.582479392671887


ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'head': object 'SCSRD' not found


In [18]:
# CCInx
CCInxA <- run_CCInx(sampleA1, top = 208)
CCInxB <- run_CCInx(sampleB1, top = 208)
CCInxC <- run_CCInx(sampleC1, top = 208)
CCInxD <- run_CCInx(sampleD1, top = 208)

Loading required package: scClustViz

-- Calculating cluster-wise gene statistics --

Scaling node weights per cell type...

Building node metadata...

Building edge list...



[1] "Total predicted L-R pairs: 5419"
[1] "Choose top L-R pairs: 208"


-- Calculating cluster-wise gene statistics --

Scaling node weights per cell type...

Building node metadata...

Building edge list...



[1] "Total predicted L-R pairs: 6287"
[1] "Choose top L-R pairs: 208"


-- Calculating cluster-wise gene statistics --

Scaling node weights per cell type...

Building node metadata...

Building edge list...



[1] "Total predicted L-R pairs: 5444"
[1] "Choose top L-R pairs: 208"


-- Calculating cluster-wise gene statistics --

Scaling node weights per cell type...

Building node metadata...

Building edge list...



[1] "Total predicted L-R pairs: 5950"
[1] "Choose top L-R pairs: 208"


In [19]:
head(CCInxA)
head(CCInxB)
head(CCInxC)
head(CCInxD)

,source,target,ligand,receptor,edgeWeight
,<chr>,<chr>,<chr>,<chr>,<chr>
1,AT2,Fibroblast,ITGB6,ITGB1,1
2,AT1,Fibroblast,ITGB6,ITGB1,1
3,AT1,AT2,ITGB1,ITGB6,0.987788248680122
4,AT1,AT1,ITGB1,ITGB6,0.987788248680122
5,AT1,AT1,ITGB6,ITGB1,0.987788248680122
6,AT2,Monocyte,ITGB6,ITGB1,0.987206853721975


,source,target,ligand,receptor,edgeWeight
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Endothelial,Monocyte,CCL2,IL1B,1
2,AT2,Monocyte,ITGB1,IL1B,1
3,AT2,AT2,ITGB1,ITGB6,0.995656734127857
4,AT2,AT2,ITGB6,ITGB1,0.995656734127857
5,Fibroblast,Monocyte,CCL2,IL1B,0.982470893274612
6,AT2,AT2,ITGB1,TGFBR2,0.979857454325993


,source,target,ligand,receptor,edgeWeight
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Endothelial,Mast,IL33,IL1RL1,1
2,Endothelial,Monocyte,IL33,IL1B,1
3,AT2,Endothelial,ITGB6,ITGB1,0.988816200487422
4,AT1,Endothelial,ITGB6,ITGB1,0.988816200487422
5,Endothelial,Monocyte,ITGB1,IL1B,0.988816200487422
6,AT1,Endothelial,TGFBR2,CTGF,0.97781005272738


,source,target,ligand,receptor,edgeWeight
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Mast,Monocyte,IL1RL1,IL1RAP,1
2,Endothelial,Monocyte,IL1R1,IL1RAP,1
3,Fibroblast,Tcell,CTGF,TGFB1,1
4,AT2,Tcell,ITGB6,TGFB1,1
5,AT1,Tcell,ITGB6,TGFB1,1
6,Endothelial,Tcell,TGFBR2,TGFB1,0.998704907678036


In [20]:
# scMLnet
scMLnetA <- run_scMLnet(sampleA2)

[1] "check table cell:"
[1] 10413     2
[1] "Rec Cluster:"
[1] "Fibroblast"
[1] "Lig Cluster:"
[1] "Macrophage"
[1] "p val:"
[1] 0.05
[1] "logfc:"
[1] 0.15
[1] "get High Exp Gene in Fibroblast"
[1] "Fibroblast:147"
[1] "gene:121"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:35"
[1] "-----------------------"
[1] "get High Exp Gene in Macrophage"
[1] "Macrophage:5892"
[1] "gene:121"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:21"
[1] "-----------------------"
[1] "Lig_Rec Num:6"
[1] "TF_Target Num:0"
[1] "XZRec_XZTF Num:1485"
[1] "Rec common in LigRec and RecTF:"
[1] "TF common in RecTF and TFTar:"
Error in RunMLnet(mtx, annfile, RecClu, LigClu, pval, logfc, LigRecLib,  : 
  error:RecTF and TFTar don't have the same TF!
[1] "check table cell:"
[1] 10413     2
[1] "Rec Cluster:"
[1] "AT2"
[1] "Lig Cluster:"
[1] "Macrophage"
[1] "p val:"
[1] 0.05
[1] "logfc:"
[1] 0.15
[1] "get High Exp Gene in AT2"
[1] "AT2:2293"
[1] "gene:121"
[1] "lo

In [21]:
scMLnetB <- run_scMLnet(sampleB2)

[1] "check table cell:"
[1] 20627     2
[1] "Rec Cluster:"
[1] "AT2"
[1] "Lig Cluster:"
[1] "Endothelial"
[1] "p val:"
[1] 0.05
[1] "logfc:"
[1] 0.15
[1] "get High Exp Gene in AT2"
[1] "AT2:1548"
[1] "gene:121"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:43"
[1] "-----------------------"
[1] "get High Exp Gene in Endothelial"
[1] "Endothelial:2553"
[1] "gene:121"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:18"
[1] "-----------------------"
[1] "Lig_Rec Num:0"
[1] "TF_Target Num:2"
[1] "XZRec_XZTF Num:3795"
Error in seq.default(1, length(PairList), 1) : 
  wrong sign in 'by' argument
[1] "check table cell:"
[1] 20627     2
[1] "Rec Cluster:"
[1] "Macrophage"
[1] "Lig Cluster:"
[1] "Endothelial"
[1] "p val:"
[1] 0.05
[1] "logfc:"
[1] 0.15
[1] "get High Exp Gene in Macrophage"
[1] "Macrophage:7622"
[1] "gene:121"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:38"
[1] "-----------------------"
[1] "get High Ex

Warning message in if (!netList[[i]] %in% NA) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (!netList[[i]] %in% NA) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (!netList[[i]] %in% NA) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (!netList[[i]] %in% NA) {:
“the condition has length > 1 and only the first element will be used”


[1] "Total predicted L-R pairs: 9"


In [22]:
scMLnetC <- run_scMLnet(sampleC2)

[1] "check table cell:"
[1] 28778     2
[1] "Rec Cluster:"
[1] "AT1"
[1] "Lig Cluster:"
[1] "AT2"
[1] "p val:"
[1] 0.05
[1] "logfc:"
[1] 0.15
[1] "get High Exp Gene in AT1"
[1] "AT1:96"
[1] "gene:121"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:10"
[1] "-----------------------"
[1] "get High Exp Gene in AT2"
[1] "AT2:3225"
[1] "gene:121"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:12"
[1] "-----------------------"
[1] "Lig_Rec Num:3"
[1] "TF_Target Num:0"
Error in seq.default(1, length(XZRec), 1) : wrong sign in 'by' argument
[1] "check table cell:"
[1] 28778     2
[1] "Rec Cluster:"
[1] "Endothelial"
[1] "Lig Cluster:"
[1] "AT2"
[1] "p val:"
[1] 0.05
[1] "logfc:"
[1] 0.15
[1] "get High Exp Gene in Endothelial"
[1] "Endothelial:3496"
[1] "gene:121"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:34"
[1] "-----------------------"
[1] "get High Exp Gene in AT2"
[1] "AT2:3225"
[1] "gene:121"
[1] "logfc.thresho

In [23]:
scMLnetD <- run_scMLnet(sampleD2)

[1] "check table cell:"
[1] 78430     2
[1] "Rec Cluster:"
[1] "Monocyte"
[1] "Lig Cluster:"
[1] "Macrophage"
[1] "p val:"
[1] 0.05
[1] "logfc:"
[1] 0.15
[1] "get High Exp Gene in Monocyte"
[1] "Monocyte:6793"
[1] "gene:119"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:6"
[1] "-----------------------"
[1] "get High Exp Gene in Macrophage"
[1] "Macrophage:60483"
[1] "gene:119"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:56"
[1] "-----------------------"
[1] "Lig_Rec Num:2"
[1] "TF_Target Num:0"
Error in seq.default(1, length(XZRec), 1) : wrong sign in 'by' argument
[1] "check table cell:"
[1] 78430     2
[1] "Rec Cluster:"
[1] "Tcell"
[1] "Lig Cluster:"
[1] "Macrophage"
[1] "p val:"
[1] 0.05
[1] "logfc:"
[1] 0.15
[1] "get High Exp Gene in Tcell"
[1] "Tcell:5088"
[1] "gene:119"
[1] "logfc.threshold:"
[1] 0.15
[1] "do T-test"
[1] "find high gene num:8"
[1] "-----------------------"
[1] "get High Exp Gene in Macrophage"
[1] "Macrophage

Warning message in if (!netList[[i]] %in% NA) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (!netList[[i]] %in% NA) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (!netList[[i]] %in% NA) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (!netList[[i]] %in% NA) {:
“the condition has length > 1 and only the first element will be used”


[1] "Total predicted L-R pairs: 8"


In [24]:
head(scMLnetA)
head(scMLnetB)
head(scMLnetC)
head(scMLnetD)

<0 x 0 matrix>

,source,target,ligrec
,<chr>,<chr>,<chr>
1,Monocyte,AT2,IL1A_IL1R1
2,Monocyte,AT2,IL1B_IL1R1
3,Monocyte,Fibroblast,IL1A_IL1R1
4,Monocyte,Fibroblast,IL1B_IL1R1
5,Mast,Macrophage,CCL4_CCR5
6,Mast,Macrophage,CCL4_CCR1


<0 x 0 matrix>

,source,target,ligrec
,<chr>,<chr>,<chr>
1,Monocyte,Fibroblast,IL1B_IL1R1
2,Monocyte,Endothelial,IL1B_IL1R1
3,Tcell,Fibroblast,TGFB1_ITGAV
4,Tcell,Fibroblast,TGFB1_TGFBR1
5,Tcell,Fibroblast,TGFB1_ITGB6
6,Tcell,Fibroblast,TGFB1_TGFBR3


## 3. SSP Calculation

In [25]:
# load SSP calculation functions
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/CellChat_SSP_calculation.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/iTalk_SSP_calculation.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/SingleCellSignalR_SSP_calculation.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/CCInx_SSP_calculation.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/scMLnet_SSP_calculation.R")

In [26]:
CellChat_SSPA <- run_CellChat_SSP(CellChatA,sampleA,gold)
iTalk_SSPA <- run_iTalk_SSP(iTalkA,sampleA,gold)
SingleCellSignalR_SSPA <- run_SingleCellSignalR_SSP(SCSRA,sampleA,gold)
CCInx_SSPA <- run_CCInx_SSP(CCInxA,sampleA,gold)

In [27]:
CellChat_SSPB <- run_CellChat_SSP(CellChatB,sampleB,gold)
iTalk_SSPB <- run_iTalk_SSP(iTalkB,sampleB,gold)
SingleCellSignalR_SSPB <- run_SingleCellSignalR_SSP(SCSRB,sampleB,gold)
CCInx_SSPB <- run_CCInx_SSP(CCInxB,sampleB,gold)
scMLnet_SSPB <- run_scMLnet_SSP(scMLnetB,sampleB,gold)

In [28]:
CellChat_SSPC <- run_CellChat_SSP(CellChatC,sampleC,gold)
iTalk_SSPC <- run_iTalk_SSP(iTalkC,sampleC,gold)
SingleCellSignalR_SSPC <- run_SingleCellSignalR_SSP(SCSRC,sampleC,gold)
CCInx_SSPC <- run_CCInx_SSP(CCInxC,sampleC,gold)

In [29]:
CellChat_SSPD <- run_CellChat_SSP(CellChatD,sampleD,gold)
iTalk_SSPD <- run_iTalk_SSP(iTalkD,sampleD,gold)
CCInx_SSPD <- run_CCInx_SSP(CCInxD,sampleD,gold)
scMLnet_SSPD <- run_scMLnet_SSP(scMLnetD,sampleD,gold)

In [30]:
cbind(CellChat_SSPA,CellChat_SSPB,CellChat_SSPC,CellChat_SSPD)

CellChat_SSPA CellChat_SSPB CellChat_SSPC CellChat_SSPD
Accuracy    1             0.2857143     0.5           0.6          
Sensitivity 0.004166667   0.008438819   0.01666667    0.03846154   
Specificity 1             0.9974962     0.998         0.996991

In [31]:
cbind(iTalk_SSPA,iTalk_SSPB,iTalk_SSPC,iTalk_SSPD)

iTalk_SSPA iTalk_SSPB iTalk_SSPC iTalk_SSPD
Accuracy    0.3365385  0.2788462  0.2644231  0.375     
Sensitivity 0.3465347  0.2815534  0.2696078  0.3786408 
Specificity 0.9493392  0.9446903  0.943459   0.9525027

In [32]:
cbind(SingleCellSignalR_SSPA,SingleCellSignalR_SSPB,SingleCellSignalR_SSPC)

SingleCellSignalR_SSPA SingleCellSignalR_SSPB
Accuracy    0.8333333              0.3333333             
Sensitivity 0.02525253             0.005050505           
Specificity 0.9995524              0.9991047             
            SingleCellSignalR_SSPC
Accuracy    0.5833333             
Sensitivity 0.03535354            
Specificity 0.9977619

In [33]:
cbind(CCInx_SSPA,CCInx_SSPB,CCInx_SSPC,CCInx_SSPD)

CCInx_SSPA CCInx_SSPB CCInx_SSPC CCInx_SSPD
Accuracy    0.01923077 0.04807692 0.02403846 0.01442308
Sensitivity 0.02116402 0.05235602 0.02645503 0.01587302
Specificity 0.9407321  0.9425921  0.9411253  0.9406485

In [34]:
cbind(scMLnet_SSPB,scMLnet_SSPD)

scMLnet_SSPB scMLnet_SSPD
Accuracy    0.1111111    0.5         
Sensitivity 0.005291005  0.02116402  
Specificity 0.9963336    0.9981643